In [0]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=1qk01iDtHSV-2eYlaPFZCu16RZ9BRSfNN'

In [4]:
fluff, id = link.split('=')
print (id)

1qk01iDtHSV-2eYlaPFZCu16RZ9BRSfNN


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('falldeteciton.csv')  
data = pd.read_csv('falldeteciton.csv')

In [6]:
data.shape

(16382, 7)

In [7]:
data.head
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16382 entries, 0 to 16381
Data columns (total 7 columns):
ACTIVITY       16382 non-null int64
TIME           16382 non-null float64
SL             16382 non-null float64
EEG            16382 non-null float64
BP             16382 non-null int64
HR             16382 non-null int64
CIRCLUATION    16382 non-null int64
dtypes: float64(3), int64(4)
memory usage: 896.0 KB


In [0]:
x=data.drop('ACTIVITY',axis=1)
y=data['ACTIVITY']

In [12]:
data.isnull().sum()

ACTIVITY       0
TIME           0
SL             0
EEG            0
BP             0
HR             0
CIRCLUATION    0
dtype: int64

In [0]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size =0.2 ,stratify=y,random_state=42)


In [17]:
mlpClassifier = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(10, 5), random_state=1)
mlpClassifier.fit(X_train, y_train)
score = mlpClassifier.score(X_test, y_test)
print(score)

0.22001830942935613


In [18]:
import tensorflow as tf
from tensorflow import keras

In [0]:
tf.set_random_seed(42)
np.random.seed(42)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [21]:
sc.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [24]:
model_MLP = keras.models.Sequential()

model_MLP.add(keras.layers.Dense(units=21, activation='relu',input_shape= X_train.shape[1:]))

model_MLP.add(keras.layers.Dense(units=6, activation='softmax'))

model_MLP.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 21)                147       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 132       
Total params: 279
Trainable params: 279
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_MLP.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [26]:
model_MLP.fit(x=X_train_std, y=y_train, validation_split=0.1, epochs=50,batch_size=32)

Train on 11794 samples, validate on 1311 samples
Epoch 1/50
11794/11794 [==============================] - 1s 65us/sample - loss: 1.6579 - acc: 0.2885 - val_loss: 1.5695 - val_acc: 0.3394
Epoch 2/50
11794/11794 [==============================] - 1s 48us/sample - loss: 1.5801 - acc: 0.3289 - val_loss: 1.5433 - val_acc: 0.3494
Epoch 3/50
11794/11794 [==============================] - 1s 46us/sample - loss: 1.5605 - acc: 0.3406 - val_loss: 1.5301 - val_acc: 0.3616
Epoch 4/50
11794/11794 [==============================] - 1s 45us/sample - loss: 1.5479 - acc: 0.3448 - val_loss: 1.5192 - val_acc: 0.3654
Epoch 5/50
11794/11794 [==============================] - 1s 46us/sample - loss: 1.5386 - acc: 0.3499 - val_loss: 1.5128 - val_acc: 0.3837
Epoch 6/50
11794/11794 [==============================] - 1s 44us/sample - loss: 1.5309 - acc: 0.3499 - val_loss: 1.5038 - val_acc: 0.3814
Epoch 7/50
11794/11794 [==============================] - 1s 44us/sample - loss: 1.5240 - acc: 0.3561 - val_loss: 1.4

In [27]:
test_loss, test_accuracy = model_MLP.evaluate(x=X_test_std, y=y_test)

3277/3277 [==============================] - 0s 25us/sample - loss: 1.3643 - acc: 0.4654


In [28]:
print(test_loss,test_accuracy)

1.3643043023040464 0.46536466
